In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
#Open listing files
listings_file = "Resources/listings.csv"
listings_df = pd.read_csv(listings_file)
listings_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1947,Furnished Gold Coast studio #806,2153,Linda,NaN,Near North Side,41.90246,-87.63405,Entire home/apt,59,14,4,2020-03-29,0.06,7,3
1,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,55,2,178,2019-12-15,2.76,1,322
2,4505,Open to healthcare professionals only until Ju...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,117,2,394,2020-02-16,2.91,1,146
3,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,80,2,384,2020-03-08,2.92,1,335
4,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,150,4,49,2019-10-23,0.67,11,320


In [3]:
#Open neighborhood zillow file
neighborhood_file = "Resources/Neighborhood_Zhvi_AllHomes.csv"
neighborhood_df = pd.read_csv(neighborhood_file)
neighborhood_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30
0,274772,0,Northeast Dallas,Neighborhood,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,134197.0,...,328763.0,329075.0,330792.0,331337.0,332285.0,331414.0,330948.0,330181.0,329760.0,330354.0
1,112345,1,Maryvale,Neighborhood,AZ,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,NaN,...,184177.0,185737.0,187041.0,188210.0,189345.0,190887.0,193014.0,195117.0,197585.0,200393.0
2,192689,2,Paradise,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,140042.0,...,268101.0,267143.0,266860.0,267374.0,268508.0,269481.0,270883.0,272056.0,274457.0,276646.0
3,270958,3,Upper West Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,247356.0,...,1245653.0,1231122.0,1218079.0,1211863.0,1214994.0,1223361.0,1223522.0,1218477.0,1207736.0,1208018.0
4,118208,4,South Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,135240.0,...,509245.0,511662.0,514662.0,518097.0,521237.0,525139.0,529323.0,534713.0,540251.0,545001.0


### Transform listing DataFrame

In [4]:
listings_df.count()

id                                8520
name                              8520
host_id                           8520
host_name                         8520
neighbourhood_group                  0
neighbourhood                     8520
latitude                          8520
longitude                         8520
room_type                         8520
price                             8520
minimum_nights                    8520
number_of_reviews                 8520
last_review                       7193
reviews_per_month                 7193
calculated_host_listings_count    8520
availability_365                  8520
dtype: int64

In [6]:
# Create a filtered dataframe from specific columns (drop neighborhood_group)
listings_cols = ['id', 'name', 'host_id', 'host_name',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365']

listings_transformed= listings_df[listings_cols].copy()

# Rename the neighbourhood column into neighborhood
listings_transformed = listings_transformed.rename(columns={"neighbourhood": "neighborhood"})

# Clean the data by dropping duplicates and setting the index
listings_transformed.drop_duplicates("id", inplace=True)
listings_transformed.set_index("id", inplace=True)

listings_transformed.head()

,name,host_id,host_name,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,,,,
1947,Furnished Gold Coast studio #806,2153,Linda,Near North Side,41.90246,-87.63405,Entire home/apt,59,14,4,2020-03-29,0.06,7,3
2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,55,2,178,2019-12-15,2.76,1,322
4505,Open to healthcare professionals only until Ju...,5775,Craig & Kathleen,South Lawndale,41.85495,-87.69696,Entire home/apt,117,2,394,2020-02-16,2.91,1,146
7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90289,-87.68182,Entire home/apt,80,2,384,2020-03-08,2.92,1,335
9811,Barbara's Hideaway - Old Town,33004,At Home Inn,Lincoln Park,41.91769,-87.63788,Entire home/apt,150,4,49,2019-10-23,0.67,11,320


### Transform neighborhood DataFrame

In [7]:
neighborhood_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30
0,274772,0,Northeast Dallas,Neighborhood,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,134197.0,...,328763.0,329075.0,330792.0,331337.0,332285.0,331414.0,330948.0,330181.0,329760.0,330354.0
1,112345,1,Maryvale,Neighborhood,AZ,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,NaN,...,184177.0,185737.0,187041.0,188210.0,189345.0,190887.0,193014.0,195117.0,197585.0,200393.0
2,192689,2,Paradise,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,140042.0,...,268101.0,267143.0,266860.0,267374.0,268508.0,269481.0,270883.0,272056.0,274457.0,276646.0
3,270958,3,Upper West Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,247356.0,...,1245653.0,1231122.0,1218079.0,1211863.0,1214994.0,1223361.0,1223522.0,1218477.0,1207736.0,1208018.0
4,118208,4,South Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,135240.0,...,509245.0,511662.0,514662.0,518097.0,521237.0,525139.0,529323.0,534713.0,540251.0,545001.0


In [10]:
# Create a filtered dataframe from specific columns (watchout for european date format!)
neighborhood_cols = ['RegionID', 'SizeRank', 'RegionName', 'State', 'City', 'Metro', 'CountyName', 
                     "2019-01-31","2019-02-28","2019-03-31","2019-04-30","2019-05-31",
                     "2019-06-30","2019-07-31","2019-08-31","2019-09-30","2019-10-31","2019-11-30","2019-12-31",
                    "2020-01-31","2020-02-29","2020-03-31","2020-04-30"]

neighborhood_transformed = neighborhood_df[neighborhood_cols].copy()

# Rename the column headers
neighborhood_transformed = neighborhood_transformed.rename(columns={"RegionID": "id", "RegionName": "Neighborhood"})

# Clean the data by dropping duplicates and setting the index
neighborhood_transformed.drop_duplicates("id", inplace=True)
neighborhood_transformed.set_index("id", inplace=True)


neighborhood_transformed

,SizeRank,Neighborhood,State,City,Metro,CountyName,2019-01-31,2019-02-28,2019-03-31,2019-04-30,...,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30
id,,,,,,,,,,,,,,,,,,,,,
274772,0,Northeast Dallas,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,324637.0,326062.0,327092.0,327497.0,...,328763.0,329075.0,330792.0,331337.0,332285.0,331414.0,330948.0,330181.0,329760.0,330354.0
112345,1,Maryvale,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,179304.0,180222.0,181021.0,181837.0,...,184177.0,185737.0,187041.0,188210.0,189345.0,190887.0,193014.0,195117.0,197585.0,200393.0
192689,2,Paradise,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,267475.0,268947.0,269033.0,269480.0,...,268101.0,267143.0,266860.0,267374.0,268508.0,269481.0,270883.0,272056.0,274457.0,276646.0
270958,3,Upper West Side,NY,New York,New York-Newark-Jersey City,New York County,1290079.0,1278523.0,1280139.0,1266046.0,...,1245653.0,1231122.0,1218079.0,1211863.0,1214994.0,1223361.0,1223522.0,1218477.0,1207736.0,1208018.0
118208,4,South Los Angeles,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,501378.0,501129.0,500697.0,501828.0,...,509245.0,511662.0,514662.0,518097.0,521237.0,525139.0,529323.0,534713.0,540251.0,545001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107413,17008,Weatherly Heights,AL,Huntsville,Huntsville,Madison County,196492.0,194940.0,194534.0,194792.0,...,197230.0,197816.0,198844.0,199879.0,201312.0,203808.0,206727.0,210711.0,214471.0,218473.0
117927,17008,Santa Rita,CA,Salinas,Salinas,Monterey County,424484.0,424966.0,425219.0,426569.0,...,429375.0,430777.0,430652.0,430944.0,431372.0,432134.0,434540.0,435296.0,436818.0,436868.0
122375,17008,Green Valley,DE,Newark,Philadelphia-Camden-Wilmington,New Castle County,203848.0,204749.0,206096.0,206161.0,...,206160.0,206546.0,206738.0,206565.0,206923.0,207921.0,208717.0,209470.0,209763.0,210848.0


In [11]:
#Create a new column for average price
neighborhood_transformed["avgprice"]=round(neighborhood_transformed.iloc[:,6:22].mean(axis=1), 2)

neighborhood_transformed.head()

,SizeRank,Neighborhood,State,City,Metro,CountyName,2019-01-31,2019-02-28,2019-03-31,2019-04-30,...,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,avgprice
id,,,,,,,,,,,,,,,,,,,,,
274772,0,Northeast Dallas,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,324637.0,326062.0,327092.0,327497.0,...,329075.0,330792.0,331337.0,332285.0,331414.0,330948.0,330181.0,329760.0,330354.0,329124.81
112345,1,Maryvale,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,179304.0,180222.0,181021.0,181837.0,...,185737.0,187041.0,188210.0,189345.0,190887.0,193014.0,195117.0,197585.0,200393.0,187454.75
192689,2,Paradise,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,267475.0,268947.0,269033.0,269480.0,...,267143.0,266860.0,267374.0,268508.0,269481.0,270883.0,272056.0,274457.0,276646.0,269702.75
270958,3,Upper West Side,NY,New York,New York-Newark-Jersey City,New York County,1290079.0,1278523.0,1280139.0,1266046.0,...,1231122.0,1218079.0,1211863.0,1214994.0,1223361.0,1223522.0,1218477.0,1207736.0,1208018.0,1239603.81
118208,4,South Los Angeles,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,501378.0,501129.0,500697.0,501828.0,...,511662.0,514662.0,518097.0,521237.0,525139.0,529323.0,534713.0,540251.0,545001.0,516624.44


In [12]:
#Select everything that is Cook County and IL
neighborhood_cook=neighborhood_transformed.loc[(neighborhood_transformed["State"] == "IL") & 
                            (neighborhood_transformed["CountyName"] == "Cook County")]
neighborhood_cook.head()

,SizeRank,Neighborhood,State,City,Metro,CountyName,2019-01-31,2019-02-28,2019-03-31,2019-04-30,...,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,avgprice
id,,,,,,,,,,,,,,,,,,,,,
269592,53,Logan Square,IL,Chicago,Chicago-Naperville-Elgin,Cook County,402533.0,400494.0,398697.0,398381.0,...,399525.0,399630.0,399570.0,399564.0,399327.0,399902.0,402142.0,405063.0,407848.0,400567.00
403117,88,Little Village,IL,Chicago,Chicago-Naperville-Elgin,Cook County,147991.0,148379.0,149423.0,150563.0,...,150893.0,151254.0,151479.0,151491.0,150889.0,150736.0,149629.0,148383.0,146552.0,149908.31
403169,106,West Rogers Park,IL,Chicago,Chicago-Naperville-Elgin,Cook County,300966.0,299935.0,299100.0,299010.0,...,299357.0,299427.0,300033.0,300241.0,300644.0,301108.0,302396.0,303772.0,304590.0,300483.50
403120,144,South Austin,IL,Chicago,Chicago-Naperville-Elgin,Cook County,163334.0,163289.0,164094.0,165241.0,...,166645.0,166748.0,166938.0,166992.0,167316.0,168038.0,169046.0,169449.0,169386.0,166624.31
269566,154,Albany Park,IL,Chicago,Chicago-Naperville-Elgin,Cook County,321243.0,319922.0,319309.0,318606.0,...,318728.0,317756.0,316704.0,315771.0,315963.0,316232.0,316220.0,315948.0,316181.0,317760.00


## Merge on neighborhood

In [13]:
#Rename column to merge on so that they match
neighborhood_cook = neighborhood_cook.rename(columns={"Neighborhood": "neighborhood"})

#merge df
merged_df= pd.merge(listings_transformed, neighborhood_cook, on="neighborhood", left_index=True)
merged_df.head()

,name,host_id,host_name,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,...,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,avgprice
id,,,,,,,,,,,,,,,,,,,,,
269586,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,55,2,178,...,394755.0,394109.0,393642.0,392811.0,392230.0,391849.0,392167.0,392182.0,390609.0,392340.94
269586,3 Comforts of Cooperative Living,2140,Lois,Hyde Park,41.79612,-87.59261,Private room,35,2,44,...,394755.0,394109.0,393642.0,392811.0,392230.0,391849.0,392167.0,392182.0,390609.0,392340.94
269586,Historic Chicago Home - Hyde Park,1517871,Jon,Hyde Park,41.80086,-87.58948,Entire home/apt,195,7,8,...,394755.0,394109.0,393642.0,392811.0,392230.0,391849.0,392167.0,392182.0,390609.0,392340.94
269586,Spacious Stylish One Bedroom Apt In HydePark,9192868,Rebecca,Hyde Park,41.79984,-87.59465,Entire home/apt,101,2,337,...,394755.0,394109.0,393642.0,392811.0,392230.0,391849.0,392167.0,392182.0,390609.0,392340.94
269586,Musician's Quarters,9601147,Chester,Hyde Park,41.79963,-87.59477,Entire home/apt,115,3,45,...,394755.0,394109.0,393642.0,392811.0,392230.0,391849.0,392167.0,392182.0,390609.0,392340.94


In [14]:
# Drop all columns with dates, to keep just average price column
merged_df=merged_df.drop(["2019-01-31","2019-02-28","2019-03-31","2019-04-30","2019-05-31",
                     "2019-06-30","2019-07-31","2019-08-31","2019-09-30","2019-10-31","2019-11-30","2019-12-31",
                    "2020-01-31","2020-02-29","2020-03-31","2020-04-30"], axis=1)
merged_df.head()

,name,host_id,host_name,neighborhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,SizeRank,State,City,Metro,CountyName,avgprice
id,,,,,,,,,,,,,,,,,,,,
269586,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,55,2,178,2019-12-15,2.76,1,322,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
269586,3 Comforts of Cooperative Living,2140,Lois,Hyde Park,41.79612,-87.59261,Private room,35,2,44,2020-02-14,0.66,5,268,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
269586,Historic Chicago Home - Hyde Park,1517871,Jon,Hyde Park,41.80086,-87.58948,Entire home/apt,195,7,8,2018-09-17,0.08,2,348,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
269586,Spacious Stylish One Bedroom Apt In HydePark,9192868,Rebecca,Hyde Park,41.79984,-87.59465,Entire home/apt,101,2,337,2020-03-10,4.25,1,332,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94
269586,Musician's Quarters,9601147,Chester,Hyde Park,41.79963,-87.59477,Entire home/apt,115,3,45,2019-08-05,0.59,1,329,692,IL,Chicago,Chicago-Naperville-Elgin,Cook County,392340.94


### Create database connection

In [16]:
connection_string = "postgres:postgres@localhost:5432/Airbnb"
engine = create_engine(f'postgresql://{connection_string}')

In [17]:
# Confirm tables
engine.table_names()

['listing', 'neighborhood', 'listing_neighborhood']

### Load DataFrames into database

In [18]:
listings_transformed.to_sql(name='listing', con=engine, if_exists='append', index=True)

In [19]:
neighborhood_transformed.to_sql(name='neighborhood', con=engine, if_exists='append', index=True)

In [21]:
merged_df.to_sql(name='listing_neighborhood', con=engine, if_exists='append', index=True)